In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

pd.options.mode.chained_assignment = None
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

In [79]:
# Iterate through each action and determine the lineup (only changes on subtitution actions)
def generate_lineups(df, team, starters):
    starters_copy = starters.copy()
    current = starters

    lineups = [", ".join(starters)]
    
    for x in range(1, len(df)):
        if (df.Opponent[x] == 'North Central (Ill.)') and (df.Time[x] == '07:53') and (df.Half[x] == 2):
            current = "Lindsey Carter, Grace Hynes, Bella Alfaro, Marissa Powe, Isabelle Herrera".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'North Central (Ill.)') and (df.Time[x] == '04:52') and (df.Half[x] == 4):
            current = "Grace Hynes, Ashley Gao, Ellie Gross, Marissa Powe, Lindsey Carter".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'North Park') and (df.Time[x] == '07:14') and (df.Half[x] == 2):
            current = "Ellie Gross, Grace Hynes, Sophia North, Marissa Powe, Ashley Gao".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Colorado College') and (df.Time[x] == '04:29') and (df.Half[x] == 3):
            current = "Grace Hynes, Ashley Gao, Sophia North, Lindsey Carter, Bella Alfaro".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Colorado College') and (df.Time[x] == '16:23') and (df.Half[x] == 2):
            current = "Alec Bryan, Eamonn Kenah, Joe Berry, Arrish Bhandal, Thomas Kurowski".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Eureka') and (df.Time[x] == '05:00') and (df.Half[x] == 2):
            current = "Marissa Powe, Grace Hynes, Ashley Gao, Bella Alfaro, Sophia North".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Wheaton (Ill.)') and (df.Time[x] == '07:06') and (df.Half[x] == 4):
            current = "Ellie Gross, Marissa Powe, Sophia North, Mallory Brodnik, Bella Alfaro".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Wis. Lutheran') and (df.Time[x] == '09:59') and (df.Half[x] == 4):
            current = "Ellie Gross, Mallory Brodnik, Grace Hynes, Sophia North, Marissa Powe".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Ill. Wesleyan') and (df.Time[x] == '02:26') and (df.Half[x] == 3):
            current = "Elliot Paschal, Bryce Hopkins, Thomas Kurowski, Alex Battist, Joe Berry".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'New York U.') and (df.Time[x] == '15:13') and (df.Half[x] == 2):
            current = "Thomas Kurowski, Bryce Hopkins, Eamonn Kenah, Joe Berry, Jackson Piotrowski".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'New York U.') and (df.Time[x] == '06:06') and (df.Half[x] == 2):
            current = "Eamonn Kenah, Joe Berry, Thomas Kurowski, Jackson Piotrowski, Bryce Hopkins".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Brandeis') and (df.Time[x] == '04:00') and (df.Half[x] == 4):
            current = "Bella Alfaro, Marissa Powe, Ellie Gross, Mallory Brodnik, Grace Hynes".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Case Western Reserve') and (df.Time[x] == '14:01') and (df.Half[x] == 2):
            current = "Bryce Hopkins, Ezra Moos, Eamonn Kenah, Elliot Paschal, Jackson Piotrowski".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Case Western Reserve') and (df.Time[x] == '09:12') and (df.Half[x] == 2):
            current = "Thomas Kurowski, Bryce Hopkins, Joe Berry, Jackson Piotrowski, Elliot Paschal".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Case Western Reserve') and (df.Time[x] == '07:25') and (df.Half[x] == 2):
            current = "Grace Hynes, Sophia North, Mallory Brodnik, Bella Alfaro, Ellie Gross".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Case Western Reserve') and (df.Time[x] == '06:51') and (df.Half[x] == 2):
            current = "Sophia North, Ashley Gao, Ellie Gross, Grace Hynes, Mallory Brodnik".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Case Western Reserve') and (df.Time[x] == '05:52') and (df.Half[x] == 3):
            current = "Ashley Gao, Lindsey Carter, Marissa Powe, Ellie Gross, Grace Hynes".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Case Western Reserve') and (df.Time[x] == '05:19') and (df.Half[x] == 3):
            current = "Ashley Gao, Lindsey Carter, Marissa Powe, Ellie Gross, Grace Hynes".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Case Western Reserve') and (df.Time[x] == '04:59') and (df.Half[x] == 3):
            current = "Ellie Gross, Ashley Gao, Sophia North, Marissa Powe, Grace Hynes".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Opponent[x] == 'Rochester (N.Y.)') and (df.Time[x] == '00:18') and (df.Half[x] == 2):
            current = "Bryce Hopkins, Eamonn Kenah, Thomas Kurowski, Skyler Twyman, Joe Berry".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Date[x] == "February 5, 2023") and (df.Time[x] == '09:36') and (df.Half[x] == 2):
            current = "Ellie Gross, Kate Gross, Sophia North, Lindsey Carter, Grace Hynes".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Date[x] == "February 5, 2023") and (df.Time[x] == '00:33') and (df.Half[x] == 4):
            current = "Grace Hynes, Mallory Brodnik, Ellie Gross, Marissa Powe, Bella Alfaro".split(", ")
            lineups.append( ", ".join(current) )
            continue
        if (df.Date[x] == "February 5, 2023") and (df.Time[x] == '19:46') and (df.Half[x] == 2):
            current = "Bryce Hopkins, Eamonn Kenah, Joe Berry, Thomas Kurowski, Arrish Bhandal".split(", ")
            lineups.append( ", ".join(current) )
            continue
        
        action = df.Action[x]
        if (df.Half[x] == 3) and df.Half[x-1] == 2: # DIFFERENT FOR TEAMS
                current = starters_copy
        
        # Lineup doesn't change if other team is responsible for action
        if (df.Team[x] != team) or (df.index[x] == 342):
            lineups.append( ", ".join(current) )
            continue
            
        # Change lineup on substitution plays
        player = extract_player(action)
        
        if ("game" in action) and (player not in current): # sub in
                current.append(player)
        elif ("bench" in action) and (player in current): # sub out
            current.remove(player)

        lineups.append( ", ".join(current) )

    return lineups

In [80]:
# Extract player name from play-by-play action
# Ex. "Foul by LAST_NAME,FIRST_NAME"
def extract_player(action):
    player_raw = [i for i in action.split() if "," in i] # Ex. ['LAST_NAME,FIRST_NAME']
    
    if len(player_raw) == 0: # Ex. team timeout
        player = ""
    else:
        player = list(reversed(player_raw[0].split(","))) # split by comma, reverse list
        player = " ".join(player).title() # reduce to one string, fix capitalization

        # outlier due to accents
        if "Kerem" in player:
            player = 'Kerem Öztürk'
        
    return player

In [81]:
# Extract attributes from HTML of play-by-play action, 
# status of Chicago team (home or away), and half (1 or 2)
def clean_row(row, chicago, half):
    time = row.find('td', {'class' : 'time'}).string

    scores = row.find_all('span', {'class' : ['v-score', 'h-score']})
    if len(scores) == 0:
        score = "0-0"
    else:
        score = '-'.join([i.string for i in scores])
        
    # Identify the team repsonsible for the action
    action_team = row.find('img')['alt']
    if "Chicago" in action_team:
        team = chicago
    else:
        if chicago == 'Home':
            team = 'Away'
        else:
            team = 'Home'
        
    try:
        action = row.find('span', {'class' : 'text'}).string.strip()
    except:
        action = row.find('strong').string # timeouts are contained in bold tags
    action = ' '.join(action.split())
    action = action.replace("\n", "")
    
    substitution = 0
    if "the" in action: # all substitution actions end in "enters the game" or "goes to the bench"
        substitution = 1

    player = extract_player(action)

    return [half, time, score, team, action, substitution, player]

In [82]:
# Scrape play-by-play data from url, return dataframe of game actions
def uaa_pbp(d, url, chicago, opponent):
    d.get(url)
    soup = BeautifulSoup(d.page_source, 'html.parser')

    half = soup.find_all('table', { 'role' : 'presentation' })
    first = half[0].find_all('tr', {'class' : 'row'})
    second = half[1].find_all('tr', {'class' : 'row'})
    
    date = soup.find_all('h1')[1].find('span').text

    first_half = [[date, chicago, opponent] + clean_row(r, chicago, 1) for r in first]
    second_half = [[date, chicago, opponent] + clean_row(r, chicago, 2) for r in second]
    
    if len(half) == 3:
        third = half[2].find_all('tr', {'class' : 'row'})
        third_quarter = [[date, chicago, opponent] + clean_row(r, chicago, 3) for r in third]
        
        game = pd.DataFrame(first_half + second_half + third_quarter, 
                            columns = ['Date', 'Chicago', 'Opponent', 'Half', 'Time', 'Score', 
                                       'Team', 'Action', 'Substitution', 'Player'])
    elif len(half) == 4:
        third = half[2].find_all('tr', {'class' : 'row'})
        fourth = half[3].find_all('tr', {'class' : 'row'})
        
        third_quarter = [[date, chicago, opponent] + clean_row(r, chicago, 3) for r in third]
        fourth_quarter = [[date, chicago, opponent] + clean_row(r, chicago, 4) for r in fourth]
        
        game = pd.DataFrame(first_half + second_half + third_quarter + fourth_quarter, 
                            columns = ['Date', 'Chicago', 'Opponent', 'Half', 'Time', 'Score', 
                                       'Team', 'Action', 'Substitution', 'Player'])
    else:
        game = pd.DataFrame(first_half + second_half, 
                            columns = ['Date', 'Chicago', 'Opponent', 'Half', 'Time', 'Score', 
                                       'Team', 'Action', 'Substitution', 'Player'])

    return game

In [83]:
# Pull points scored from each action/play
def points_scored(action):
    if "made 3" in action:
        return 3
    elif "made free" in action:
        return 1
    elif "made" in action:
        return 2
    else:
        return 0

In [84]:
# Scrape single game from box score url
def game_scraper(driver, url):
    global away_or_home
    
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # First logo tag belongs to away team : if it is Chicago's logo, then Chicago is the away team
    uc_logo = "https://cdn.prestosports.com/action/cdn/logos/rpi/137/mbkb.png"
    if uc_logo == soup.find('div', {'class' : 'team-logo'}).find('img')['src']:
        chicago = away_or_home
    else:
        chicago = away_or_home
        
    teams = [t.text.strip() for t in soup.find_all('tbody')[0].find_all('th', {'class' : 'name'})]
    teams = [" ".join(t.split(" ")[1:]) if t[0] == "#" else t for t in teams]
    opponent = [t for t in teams if "Chicago" not in t][0]
    
    chicago_players = [a.string for a in soup.find_all('a', {'class' : 'player-name'})]
    chicago_players = chicago_players[0:int(len(chicago_players) / 3)]
    
    # Scrape play-by-play data using function defined above
    pbp_url = "https://athletics.uchicago.edu" + soup.find('a', {'id' : 'pbp'})['href']
    plays = uaa_pbp(driver, pbp_url, chicago, opponent)
    
    # Add lineup column based on home/away status
    plays['Lineup'] = generate_lineups(plays, chicago, chicago_players[0:5])
    
    # Add column for points scored for each play
    plays['Points'] = plays.Action.map(points_scored)
    
    # Separate dataframe by team
    chicago = plays[(plays.Chicago == plays.Team)]
    opponent = plays[(plays.Chicago != plays.Team)]
    
    # Create new dataframe with aggregate stats
    plays_stats = pd.DataFrame(columns = ['Stats', 'Chicago', 'Opponent'])
    plays_stats['Stats'] = ['OREB', 'DREB', 'STL', 'BLK', 'PF', 'PTS']
    plays_stats['Chicago'] = [chicago[(chicago.Action.str.contains("offensive rebound"))].shape[0],
               chicago[(chicago.Action.str.contains("defensive rebound"))].shape[0],
               chicago[(chicago.Action.str.contains("Steal"))].shape[0],
               chicago[(chicago.Action.str.contains("Block"))].shape[0],
               chicago[(chicago.Action.str.contains("Foul"))].shape[0],
               chicago.Points.sum()]
    plays_stats['Opponent'] = [opponent[(opponent.Action.str.contains("offensive rebound"))].shape[0],
                opponent[(opponent.Action.str.contains("defensive rebound"))].shape[0],
                opponent[(opponent.Action.str.contains("Steal"))].shape[0],
                opponent[(opponent.Action.str.contains("Block"))].shape[0],
                opponent[(opponent.Action.str.contains("Foul"))].shape[0],
                opponent.Points.sum()]
    plays_stats.columns = ['Stats', 'Chicago', plays.Opponent[0]]
    
    
    # Scrape team stats page
    driver.get( url.split("view=")[0] + "view=teamstats" )
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Pull out names of stats and actual values
    stats = [i.text for i in soup.find_all('div', {'class': 'stats-box half'})[1].find_all('th')]
    values = [i.text.strip() for i in soup.find_all('div', {'class': 'stats-box half'})[1].find_all('td')]

    # Create new dataframe with aggregate stats
    team_stats = pd.DataFrame()
    team_stats['Stats'] = stats[3:-1]
    team_stats['Team 1'] = [values[i-1] for i in range(1, len(values), 2)]
    team_stats['Team 2'] = [values[i] for i in range(1, len(values), 2)]    
    
    if stats[1][0] == "#":
        team1 = " ".join(stats[1].split(" ")[1:])
    else:
        team1 = stats[1]
    if stats[2][0] == "#":
        team2 = " ".join(stats[2].split(" ")[1:])
    else:
        team2 = stats[2]
    team_stats.columns = ['Stats', team1, team2]
    
    # Concatenate two dataframes of aggregate stats
    df = pd.concat([team_stats, plays_stats]).reset_index(drop=True)
    
    all_stats = df.transpose().iloc[1:, :]
    all_stats = all_stats.reset_index(drop=False)
    all_stats.columns = ['Team'] + list(df.Stats)
    all_stats['Opponent'] = [t for t in teams if "Chicago" not in t][0]
    
    all_stats['Date'] = plays.Date.values[0]
    
    return [plays, all_stats]

In [85]:
plays = True

def uaa_scraper(schedule_url, num_games):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(schedule_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    games = soup.find_all('a', {'class' : 'link'})[1::2]
    game_links = ["https://athletics.uchicago.edu" + i['href'] + "?view=plays" for i in games]
    game_links = game_links[0:num_games]
    games = [game_scraper(driver, g)[0] for g in game_links] # g[0] for play-by-play
    games = pd.concat(games)
    
    if plays == True:
        current = 1
        games_list = [1]
        for i in range(1, len(games)):
            if games.Date.values[i] != games.Date.values[i-1]:
                current +=1
            games_list.append(current)

        games['Game'] = games_list
    
    driver.quit()

    return games


m_season = "https://athletics.uchicago.edu/sports/mbkb/2022-23/schedule"
w_season = "https://athletics.uchicago.edu/sports/wbkb/2022-23/schedule"

# season = uaa_scraper(m_season, 5)

In [86]:
# season.to_csv("season_plays23.csv")

In [87]:
def single_scraper(url):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    df = game_scraper(driver, url)
    driver.quit()
    
    return df

def time_to_seconds(time):
    return 60*int(time.split(":")[0]) + int(time.split(":")[1])

## Overview

In [88]:
away_or_home = "Home"
url = "https://athletics.uchicago.edu/sports/mbkb/2022-23/boxscores/20230205_h2o1.xml?view=boxscore"

df = single_scraper(url)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 109.0.5414
[WDM] - Get LATEST chromedriver version for 109.0.5414 google-chrome
[WDM] - Driver [/Users/jeremydumalig/.wdm/drivers/chromedriver/mac64/109.0.5414.74/chromedriver] found in cache
/var/folders/91/7lxj2xq13852sql471wdl0pw0000gn/T/ipykernel_73166/2013076570.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [75]:
season = pd.read_csv("season_overview23.csv").iloc[:, 1:]

season = pd.concat([season, df[1]])
# season = season[(season.index < 24)]

season['AST'] = season['Assists']
season['REB'] = season['Rebounds']
season['TOV'] = season['Turnovers']
season['PTS OFF TOV'] = season['Points off Turnovers']
season['2ND CHANCE'] = season['2nd Chance Points']
if 'Points in the Paint' in season.columns:
    season['PAINT'] = season['Points in the Paint']
if 'Fastbreak Points' in season.columns:
    season['FASTBREAK'] = season['Fastbreak Points']
season['BENCH'] = season['Bench Points']

season['FGM'] = season['Field Goal'].map(lambda s : int( s.split("-")[0] ))
season['FGA'] = season['Field Goal'].map(lambda s : int( s.split("-")[1] ))
season['3PM'] = season['3 Point'].map(lambda s : int( s.split("-")[0] ))
season['3PA'] = season['3 Point'].map(lambda s : int( s.split("-")[1] ))
season['FTM'] = season['Free Throw'].map(lambda s : int( s.split("-")[0] ))
season['FTA'] = season['Free Throw'].map(lambda s : int( s.split("-")[1] ))

season['FG%'] = 100 * round(season['FGM'] / season['FGA'], 3)
season['3P%'] = 100 * round(season['3PM'] / season['3PA'], 3)
season['FT%'] = 100 * round(season['FTM'] / season['FTA'], 3)

season['Chicago'] = (season.Team == "Chicago")

season = season[season.columns.drop(list(season.filter(regex='Unnamed')))]

season

,Team,Field Goal,Field Goal %,3 Point,3 Point %,Free Throw,Free Throw %,Rebounds,Assists,Turnovers,...,FGM,FGA,3PM,3PA,FTM,FTA,FG%,3P%,FT%,Chicago
0,Lake Forest,25-64,39.1%,9-26,34.6%,8-15,53.3%,29,15,6,...,25,64,9,26,8,15,39.1,34.6,53.3,False
1,Chicago,31-54,57.4%,8-18,44.4%,8-14,57.1%,40,19,12,...,31,54,8,18,8,14,57.4,44.4,57.1,True
2,Chicago,15-54,27.8%,8-34,23.5%,12-19,63.2%,44,7,16,...,15,54,8,34,12,19,27.8,23.5,63.2,True
3,Albion,18-56,32.1%,0-10,0%,6-11,54.5%,41,6,17,...,18,56,0,10,6,11,32.1,0.0,54.5,False
4,Chicago,28-64,43.8%,4-18,22.2%,4-5,80.0%,39,9,9,...,28,64,4,18,4,5,43.8,22.2,80.0,True
5,Mt. St. Joseph,21-51,41.2%,8-23,34.8%,9-11,81.8%,30,11,10,...,21,51,8,23,9,11,41.2,34.8,81.8,False
6,Colorado College,28-69,40.6%,5-17,29.4%,8-14,57.1%,39,10,10,...,28,69,5,17,8,14,40.6,29.4,57.1,False
7,Chicago,24-51,47.1%,9-25,36.0%,10-12,83.3%,35,14,20,...,24,51,9,25,10,12,47.1,36.0,83.3,True
8,Colby,24-54,44.4%,13-30,43.3%,12-13,92.3%,30,16,12,...,24,54,13,30,12,13,44.4,43.3,92.3,False
9,Chicago,22-55,40.0%,5-17,29.4%,12-20,60.0%,38,10,15,...,22,55,5,17,12,20,40.0,29.4,60.0,True


In [76]:
season.drop_duplicates().to_csv("season_overview23.csv")

## Plays

In [72]:
# WBB
game = df[0]

current = 1
quarters = [1]
for i in range(1, len(game)):
    if time_to_seconds(game.Time.values[i]) > time_to_seconds(game.Time.values[i-1]):
        current += 1
    quarters.append(current)
    
game['Half'] = quarters

season_plays = pd.read_csv("w_season_plays23.csv").iloc[:, 1:]
season_plays = season_plays[(season_plays.Game != 20)]
season_plays = pd.concat([season_plays, game])

current = 1
games = [1]
for i in range(1, len(season_plays)):
    if season_plays.Date.values[i] != season_plays.Date.values[i-1]:
        current +=1
    games.append(current)
    
season_plays['Game'] = games

season_plays

,Date,Chicago,Opponent,Half,Time,Score,Team,Action,Substitution,Player,Lineup,Points,Game
0,"November 12, 2022",Away,North Central (Ill.),1,09:51,0-0,Away,"HYNES,GRACE missed layup",0,Grace Hynes,"Mallory Brodnik, Marissa Powe, Ellie Gross, Gr...",0,1
1,"November 12, 2022",Away,North Central (Ill.),1,09:45,0-0,Home,"MCCLURE,MEGAN defensive rebound",0,Megan Mcclure,"Mallory Brodnik, Marissa Powe, Ellie Gross, Gr...",0,1
2,"November 12, 2022",Away,North Central (Ill.),1,09:33,0-0,Home,"SMITH,MITRESE missed jump shot",0,Mitrese Smith,"Mallory Brodnik, Marissa Powe, Ellie Gross, Gr...",0,1
3,"November 12, 2022",Away,North Central (Ill.),1,09:29,0-0,Away,"POWE,MARISSA defensive rebound",0,Marissa Powe,"Mallory Brodnik, Marissa Powe, Ellie Gross, Gr...",0,1
4,"November 12, 2022",Away,North Central (Ill.),1,09:19,0-0,Away,"BRODNIK,MALLORY missed layup",0,Mallory Brodnik,"Mallory Brodnik, Marissa Powe, Ellie Gross, Gr...",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,"February 5, 2023",Home,Rochester (N.Y.),4,00:12,75-87,Home,"GROSS,ELLIE made free throw",0,Ellie Gross,"Grace Hynes, Mallory Brodnik, Ellie Gross, Mar...",1,20
447,"February 5, 2023",Home,Rochester (N.Y.),4,00:12,75-87,Home,"NORTH,SOPHIA enters the game",1,Sophia North,"Grace Hynes, Mallory Brodnik, Ellie Gross, Mar...",0,20
448,"February 5, 2023",Home,Rochester (N.Y.),4,00:12,75-87,Home,"ALFARO,BELLA goes to the bench",1,Bella Alfaro,"Grace Hynes, Mallory Brodnik, Ellie Gross, Mar...",0,20
449,"February 5, 2023",Home,Rochester (N.Y.),4,00:05,75-87,Away,"Turnover by MCCULLEY,CALLIE",0,Callie Mcculley,"Grace Hynes, Mallory Brodnik, Ellie Gross, Mar...",0,20


In [73]:
season_plays.to_csv("w_season_plays23.csv")

In [89]:
# MBB
season_plays = pd.read_csv("season_plays23.csv")
season_plays = season_plays[(season_plays.Date != "February 5, 2023")]

season_plays = pd.concat([season_plays, df[0]])

current = 1
games_list = [1]
for i in range(1, len(season_plays)):
    if season_plays.Date.values[i] != season_plays.Date.values[i-1]:
        current +=1
    games_list.append(current)

season_plays['Game'] = games_list

season_plays = season_plays[season_plays.columns.drop(list(season_plays.filter(regex='Unnamed')))]

season_plays

,Date,Chicago,Opponent,Half,Time,Score,Team,Action,Substitution,Player,Lineup,Points,Game
0,"November 10, 2022",Home,Lake Forest,1,19:44,2-0,Away,"SIEGIEN,FRANK made jump shot",0,Frank Siegien,"Alec Bryan, Ezra Moos, Thomas Kurowski, Elliot...",2,1
1,"November 10, 2022",Home,Lake Forest,1,19:44,2-0,Away,"Assist by BULL,ELIJAH",0,Elijah Bull,"Alec Bryan, Ezra Moos, Thomas Kurowski, Elliot...",0,1
2,"November 10, 2022",Home,Lake Forest,1,19:16,2-0,Home,"Turnover by BRYAN,ALEC",0,Alec Bryan,"Alec Bryan, Ezra Moos, Thomas Kurowski, Elliot...",0,1
3,"November 10, 2022",Home,Lake Forest,1,18:50,5-0,Away,"DEITCH,NOAH made 3-pt. jump shot",0,Noah Deitch,"Alec Bryan, Ezra Moos, Thomas Kurowski, Elliot...",3,1
4,"November 10, 2022",Home,Lake Forest,1,18:50,5-0,Away,"Assist by BULL,ELIJAH",0,Elijah Bull,"Alec Bryan, Ezra Moos, Thomas Kurowski, Elliot...",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,"February 5, 2023",Home,Rochester (N.Y.),2,00:49,69-88,Away,"Assist by CANNON,ERIC",0,Eric Cannon,"Alex Battist, Skyler Twyman, Alec Bryan, Dashi...",0,20
478,"February 5, 2023",Home,Rochester (N.Y.),2,00:20,69-88,Home,"WALKER,DASHIEL missed 3-pt. jump shot",0,Dashiel Walker,"Alex Battist, Skyler Twyman, Alec Bryan, Dashi...",0,20
479,"February 5, 2023",Home,Rochester (N.Y.),2,00:20,69-88,Away,"BASMA,AMIR defensive rebound",0,Amir Basma,"Alex Battist, Skyler Twyman, Alec Bryan, Dashi...",0,20
480,"February 5, 2023",Home,Rochester (N.Y.),2,00:13,72-88,Away,"OPREA,CORVIN made 3-pt. jump shot",0,Corvin Oprea,"Alex Battist, Skyler Twyman, Alec Bryan, Dashi...",3,20


In [90]:
season_plays.drop_duplicates().to_csv("season_plays23.csv")